In [2]:
from sympy import Matrix, init_printing, eye, shape, ones, re, Symbol, symbols, diag, zeros, ImmutableMatrix
init_printing()

In [1]:
import numpy as np
from scipy import sparse
import networkx
from fast_pagerank import pagerank
from fast_pagerank import pagerank_power
import sklearn
from sklearn.decomposition import NMF

In [3]:
from scipy import sparse

In [4]:
def power(Matrix,i):                                        # Power iteration to approximate the principal eigenvector of Matrix in i iterations
    n = shape(Matrix)[0]
    eigenvector = ones(n).col(0)                                                              
    for j in range(i):
        eigenvector = Matrix * eigenvector
    eigenvector = eigenvector / sum(eigenvector)
    return eigenvector

In [5]:
def powerscore(Matrix,m,i):                                 # Transforms Matrix to include teleportation probability m then finds the eigenvector
    n = shape(Matrix)[0]
    S = 1/n * ones(n)
    M = (1-m) * Matrix + m * S                                                      
    return power(Matrix,i)

In [6]:
def powerrecommend(Matrix,i,m,k):                           # Transforms Matrix to include teleportation probability but only to node i, then returns the eigenvector 
    n = shape(Matrix)[0]
    S = zeros(n-1,n).row_insert(i-1,ones(n).row(i-1))
    M = (1-m) * Matrix + m * S                                                         
    return power(M,k)

In [7]:
def powerrecommend_Matrix(input_Matrix,m,k):                # Computes the eigenvectors for all the recommendation matrices and puts them in a matrix. Each coloumn j represents the recommendation vector for node j. 
    n = shape(input_Matrix)[0]
    S = Matrix()
    for i in range(n):
        S = S.col_insert(i,Matrix(powerrecommend(input_Matrix,i+1,m,k)))
    return S

In [8]:
def fast_powerrecommend_Matrix(array, m, k):
    n = shape(array)[0]
    S = np.empty((0,n))
    for i in range(100): #change back to n
        personalise = np.zeros(n)
        personalise[i]=1
        S = np.vstack([S, pagerank_power(array, p=1-m, personalize=personalise, tol=1e-6)])

    return S

In [19]:
def fast_powerrecommend_Matrix2(array, m):
    n = shape(array)[0]
    S = np.zeros((n,n))
    for i in range(n): #change back to n
        personalise = np.zeros(n)
        personalise[i]=1
        S[i] = pagerank_power(array.T, p=1-m, max_iter=30, personalize=personalise, tol=1e-8) #changed array to array.T
        #with array this algorithm assumes a row-stochastic matrix!!!

    return S.T

In [10]:
j = np.zeros((0,4),int)
np.vstack([j, np.array([1,2,3,4])])


array([[1, 2, 3, 4]])

In [11]:
j = np.ones(5)
j

array([1., 1., 1., 1., 1.])

In [12]:
def fast_embedding(Matrix, m, dim):                           # Factorises the powerrecommend_Matrix into two matrices so that W * H.T = powerrecommend Matrix. 
    model = NMF(n_components=dim)                          # In the example I took inspiration from a the matrix that represents proximity between nodes = W * W.T. This way if two nodes are not close the dot-product of their embedding vectors is 0, if they are very close their dot-product is one.
    W= model.fit_transform(fast_powerrecommend_Matrix2(Matrix,m))# To somewhat approximate this property, the output is the 'average' of the two factor matrices
    H = model.components_
    #S = (W.T+H)/
    #return H.T
    return W

In [13]:
import numpy 
from numpy import array

In [14]:
import sklearn.decomposition
from sklearn.decomposition import NMF

In [15]:
def embedding(Matrix, m, k,dim):                           # Factorises the powerrecommend_Matrix into two matrices so that W * H.T = powerrecommend Matrix. 
    model = NMF(n_components=dim)                          # In the example I took inspiration from a the matrix that represents proximity between nodes = W * W.T. This way if two nodes are not close the dot-product of their embedding vectors is 0, if they are very close their dot-product is one.
    W= model.fit_transform(powerrecommend_Matrix(Matrix,m,k))# To somewhat approximate this property, the output is the 'average' of the two factor matrices
    H = model.components_
    #S = (W.T+H)/
    return H

In [1]:
def stochastic(array):
    n = shape(array)[0]
    H = numpy.empty([n,n])
    for i in range(n):
        for j in range(n):
            H[i][j] = (array[i][j])/sum((array[:,j]))
    return H
    

In [17]:
def embedding_space(array,m,k,dim): 
    K = stochastic(array.todense())
    V = embedding(K,m,k,dim).T
    return V

In [18]:
def n_power(array,i):
    n = shape(array)[0]
    eigenvector = numpy.ones(n).T                                                          
    for j in range(i):
        eigenvector = array @ eigenvector
    new_eigenvector = []
    for k in range(n):                                                                 
        new_eigenvector.append((list(eigenvector)[k])/sum(list(eigenvector)))
    return new_eigenvector
    